In [4]:
!pip install transformers
!pip install datasets
!pip install tokenizer
!pip install seqeval
!pip install transformers[torch]
!pip install seqeval

You should consider upgrading via the 'c:\users\satya\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\satya\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\satya\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\satya\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\satya\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\satya\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [7]:
!pip install datasets

You should consider upgrading via the 'c:\users\satya\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [ ]:
!pip install azureml-core
!pip install --upgrade azureml-core

In [1]:
import datasets
import numpy as np 
from transformers import BertTokenizerFast
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification

RuntimeError: Failed to import transformers.data.data_collator because of the following error (look up to see its traceback):
[WinError 127] The specified procedure could not be found. Error loading "c:\Users\satya\anaconda3\envs\tf-gpu-cuda8\lib\site-packages\torch\lib\caffe2_module_test_dynamic.dll" or one of its dependencies.

In [2]:
conll2003=datasets.load_dataset('conll2003')

Found cached dataset conll2003 (C:/Users/satya/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
conll2003['train'].features['ner_tags']

In [ ]:
conll2003['train'][0]

In [ ]:
conll2003['train'].features['ner_tags']

In [ ]:
tokenizer=BertTokenizerFast.from_pretrained('bert-base-uncased')

In [ ]:
example_text=conll2003['train'][0]
example_text

In [ ]:
tokenized_input=tokenizer(example_text['tokens'], is_split_into_words=True)

In [ ]:
tokenized_input

In [ ]:
tokenized_input['input_ids']

In [ ]:
example_text['ner_tags']

In [ ]:
tokenz=tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])

In [ ]:
tokenz

In [ ]:
for i, label in enumerate(example_text['ner_tags']):
    print(i,label)

In [ ]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs=tokenizer(examples['tokens'],truncation=False, is_split_into_words=True)
    labels=[]
    for i , label in enumerate(examples['ner_tags']):
        word_ids=tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx=None
        label_ids=[]
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx!=previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx=word_idx
        labels.append(label_ids)
    tokenized_inputs['labels']=labels 
    return tokenized_inputs   


In [ ]:
q=tokenize_and_align_labels(conll2003['train'][4:5])

In [ ]:
q

In [ ]:
import torch


In [ ]:
for token, label in zip(tokenizer.convert_ids_to_tokens(q['input_ids'][0]),q['labels'][0]):
    print(f"{token:_<40} {label}")

In [ ]:
tokenized_datasets=conll2003.map(tokenize_and_align_labels, batched=True)

In [ ]:
model=AutoModelForTokenClassification.from_pretrained('bert-base-uncased', num_labels=9)